# 3DCT高速化に向けたNumPyブロードキャストに関する検証2
## 目的 : 検証1で確かめた手法の中規模計算ベンチマーク
- 問題 : d/W(a,b,c) = d - x(a)cos(c) + y(b)sin(c) を行列演算で行いたい

In [9]:
import numpy as np
from tqdm import trange

In [7]:
a = np.arange(30)
b = np.copy(a)
c = np.copy(a)
b = b + 30
c = c + 60

a,b,c

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]),
 array([30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46,
        47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59]),
 array([60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76,
        77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89]))

In [13]:
%%timeit
abc_forloop = np.empty((a.shape[0],b.shape[0],c.shape[0]))
for i in trange(a.shape[0]):
    for j in range(b.shape[0]):
        for k in range(c.shape[0]):
            abc_forloop[i,j,k] = (a[i]*c[k]) + (b[j]*c[k])

100%|█████████████████████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 765.59it/s]


100%|█████████████████████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 832.24it/s]


53.3 ms ± 7.59 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [11]:
abc_forloop.shape

(30, 30, 30)

In [12]:
abc_forloop[1,0,0]

1860.0

In [14]:
a_re = a.reshape((a.shape[0],1))
b_re = b.reshape((b.shape[0],1))
c_re = c.reshape((1,c.shape[0]))

a_re, b_re, c_re

(array([[ 0],
        [ 1],
        [ 2],
        [ 3],
        [ 4],
        [ 5],
        [ 6],
        [ 7],
        [ 8],
        [ 9],
        [10],
        [11],
        [12],
        [13],
        [14],
        [15],
        [16],
        [17],
        [18],
        [19],
        [20],
        [21],
        [22],
        [23],
        [24],
        [25],
        [26],
        [27],
        [28],
        [29]]), array([[30],
        [31],
        [32],
        [33],
        [34],
        [35],
        [36],
        [37],
        [38],
        [39],
        [40],
        [41],
        [42],
        [43],
        [44],
        [45],
        [46],
        [47],
        [48],
        [49],
        [50],
        [51],
        [52],
        [53],
        [54],
        [55],
        [56],
        [57],
        [58],
        [59]]), array([[60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75,
         76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89]]))

In [16]:
%%timeit
ac = a_re * c_re
bc = b_re * c_re
ac_re = ac.reshape((ac.shape[0],1,ac.shape[1]))
bc_re = bc.reshape((1,bc.shape[0],bc.shape[1]))

abc = ac_re + bc_re

105 µs ± 1.11 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [19]:
np.any((abc == abc_forloop)==False)

False

- 2つの手法間に答えの違いなし
## 結果
- 30x30x30の配列においてnote PCベンチマーク
    - forloop => 53.3ms
    - 行列演算 => 105us
- バリ速でした